In [1]:
from orb_it.backend import PYOORB,FINDORB,ORBFIT
from orb_it.raiden import Orbits, getOrbHorizons, loadOrb
from orb_it.tester import runTest
import numpy as np
from astropy.time import Time
from astropy import units as u
import pandas as pd

# `orb_it` Demo

Here are the general steps of this tester,

1. Input an initial orbit state at a specified t0 time and a set of times to generate observations over.

2. Generate Ephemerides using the integrator over the set of times provided in the first step. (A user can also provide an astrometric error here to simulate real observational inaccuracies)

3. Use the integrator’s Orbit Determination function to fit the Ephemeris observations to obtain an orbital state.

4. Propagate the initial input orbit to the time of the fitted orbit state from step 3.

5. Compare the orbital state vectors from step 3 to the vectors from step 4.
---
## Importing Orbit Data
First we need to import the initial orbit data (state vector and time). We can use either `loadOrb()` to load from a local csv or pandas DataFrame or `getOrbHorizons()` to query JPL Horizons for the data. Both functions give the same orbit object.

In [2]:
#Importing Locally
orbits = loadOrb('../orbits2.csv')
# Ivezic is the first object in the table
ivezic = orbits[0]
# We can show it as a DataFrame with .df attribute
ivezic.df

,orbit_id,mjd_tdb,x,y,z,vx,vy,vz,H,G
0,202930 Ivezic (1998 SG172),59000.000801,0.568687,-2.49376,-0.182298,0.010407,0.003676,0.000483,16.65,0.15


From Horizons,

In [3]:
# Querying with JPL
# You have to specify a time for the position of the asteroid
# Using astropy.Time you can easily generate a time
t0 = Time([59000], scale="utc", format="mjd")
getOrbHorizons('202930',t0).df

,orbit_id,mjd_tdb,x,y,z,vx,vy,vz,H,G
0,202930 Ivezic (1998 SG172),59000.0,0.568687,-2.49376,-0.182298,0.010407,0.003676,0.000483,16.66,0.15


Now we have our orbit object for *202930 Ivezic (1998 SG172)* named `ivezic`. Now we can start running tests on this object.

---
## Generating Testing Parameters
Here we can generate a sample of parameters to test over. `Orb_it` can test over many different different parameters. You can generate certain parameters like observational arc and astrometric errors. We can define the observational arc using `np.arange()` which is in terms of days and the errors in terms of milliarcseconds.

In [4]:
dts = [
# Short-term, 0-30 days with an observation every 1 day
    np.arange(0, 30, 1),
    
# Weekly, 77 days with an observation every 7 days
    np.arange(0, 77, 7),
    
# Medium-term, 365 days with an observation every 10 days
    np.arange(0, 365 + 10, 10)
    
]

# 0 mas observation error, 10 mas error, and 100 mas error
errors = [0.,10.,100.]

## Using the Tester
Now that we have our parameters we can begin using the function `runTest()`. We can define the `observatory_code` which is an MPC observatory code as "500" which is the Geocenter. One can iterate over different observatory codes by passing a list of codes to the tester.

Now we will use the first time scale (`dts[0]`) and first astrometric error (`errors[0]`) we defined above.

For this section of the demo, we will be using the `FINDORB()` backend. If you want to change backends, just change `backend=` in the function call to either `PYOORB()` or `ORBFIT()`.

There is a backend specific demo after this basic usage section.

In [5]:
runTest(orbits[0],'500',dts[0],astrometric_error=errors[0],backend=FINDORB())

,orbit_id,integrator,observatory_code,arc_length [days],num_obs,epoch [mjd],delta epoch [mjd],astrometric_error [mas],delta r [km],delta v [m/s],delta x [km],delta y [km],delta z [km],delta vx [m/s],delta vy [m/s],delta vz [m/s]
0,202930 Ivezic (1998 SG172),FindOrb,500,29.0,30,59029.0008,9.997748e-07,0.0,0.000046,1.589317e-08,0.000021,-0.00004,-0.000005,-8.887649e-09,1.307467e-08,1.629640e-09


Thus `runTest()` returns the difference DataFrame which can be saved and analyzed over. See the documentation for a description of each column.

---

## Varying Time Terms
Here we can vary the time scales for each by giving `runTest()` the list of time scales `dts` we defined above.

In [6]:
runTest(orbits[0],'500',dts,astrometric_error=errors[0],backend=FINDORB())

,orbit_id,integrator,observatory_code,arc_length [days],num_obs,epoch [mjd],delta epoch [mjd],astrometric_error [mas],delta r [km],delta v [m/s],delta x [km],delta y [km],delta z [km],delta vx [m/s],delta vy [m/s],delta vz [m/s]
0,202930 Ivezic (1998 SG172),FindOrb,500,29.0,30,59029.0008,9.997748e-07,0.0,0.000046,1.589317e-08,0.000021,-0.000040,-5.057349e-06,-8.887649e-09,1.307467e-08,1.629640e-09
1,202930 Ivezic (1998 SG172),FindOrb,500,70.0,11,59070.0008,9.997748e-07,0.0,0.000009,1.037233e-09,-0.000003,0.000008,7.432393e-07,8.109717e-10,-6.457737e-10,-3.397821e-11
2,202930 Ivezic (1998 SG172),FindOrb,500,370.0,38,59370.0008,9.997748e-07,0.0,0.000280,1.257969e-08,0.000189,-0.000205,-1.919551e-05,1.233878e-08,-2.435919e-09,2.631903e-10


## Varying Over Specific Times
You can also specify specific dts for the tester to use. These must be in the format of a list of times in an `astropy.time.core.Time` object. You can make a set of specific dts for each object by passing a list of these time objects with the same lengths as the space objects being tested. Here we will test two different dts for the first two orbits in our list, *Ivezic* and *Lydia*. Thus we define our times `ob1` and `ob2`,

In [7]:
# Obtaining the first two Orbits
f2 = orbits[0:2]
# Generating the specific time object list
ob1 = t0 + [0,2,4,6,8]
ob2 = t0 + [1,3,5,7,9,11]
ob1

<Time object: scale='utc' format='mjd' value=[59000. 59002. 59004. 59006. 59008.]>

In [8]:
# Thus calling our tester
runTest(f2,'500',[ob1,ob2],FINDORB())

,orbit_id,integrator,observatory_code,arc_length [days],num_obs,epoch [mjd],delta epoch [mjd],astrometric_error [mas],delta r [km],delta v [m/s],delta x [km],delta y [km],delta z [km],delta vx [m/s],delta vy [m/s],delta vz [m/s]
0,202930 Ivezic (1998 SG172),FindOrb,500,8.0,5,59008.0008,9.997748e-07,0,0.429518,0.000020,-0.399715,-0.155965,-0.019705,5.155476e-06,-0.000020,-0.000001
1,110 Lydia (A870 HA),FindOrb,500,10.0,6,59011.0008,9.997748e-07,0,0.415987,0.000019,-0.414117,-0.017685,0.035204,-7.178541e-07,-0.000019,-0.000001


Thus we see that the specific observation times of `ob1` were tested on *Ivezic* and the times `ob2` were tested on *Lydia*.
## Varying Time and Error Terms
We can vary the time scales and error terms by passing both lists (`dts` and `errors`) to `runTest()`,

In [10]:
runTest(ivezic,'500',dts,astrometric_error=errors,backend=FINDORB())

,orbit_id,integrator,observatory_code,arc_length [days],num_obs,epoch [mjd],delta epoch [mjd],astrometric_error [mas],delta r [km],delta v [m/s],delta x [km],delta y [km],delta z [km],delta vx [m/s],delta vy [m/s],delta vz [m/s]
0,202930 Ivezic (1998 SG172),FindOrb,500,29.0,30,59029.0008,9.997748e-07,0.0,0.000046,1.589317e-08,0.000021,-0.000040,-5.057349e-06,-8.887649e-09,1.307467e-08,1.629640e-09
1,202930 Ivezic (1998 SG172),FindOrb,500,29.0,30,59029.0008,9.997748e-07,10.0,15.890394,1.369244e-02,-6.704979,14.190695,-2.484357e+00,-8.604813e-03,1.040115e-02,-2.292660e-03
2,202930 Ivezic (1998 SG172),FindOrb,500,29.0,30,59029.0008,9.997748e-07,100.0,4850.806002,1.129340e+00,2290.027335,-4249.518212,-4.771673e+02,-6.672667e-01,9.033590e-01,1.187725e-01
3,202930 Ivezic (1998 SG172),FindOrb,500,70.0,11,59070.0008,9.997748e-07,0.0,0.000009,1.037233e-09,-0.000003,0.000008,7.432393e-07,8.109717e-10,-6.457737e-10,-3.397821e-11
4,202930 Ivezic (1998 SG172),FindOrb,500,70.0,11,59070.0008,9.997748e-07,10.0,418.775421,4.217163e-02,161.180597,-383.614121,-4.726388e+01,1.094962e-02,-4.052611e-02,-4.023267e-03
5,202930 Ivezic (1998 SG172),FindOrb,500,70.0,11,59070.0008,9.997748e-07,100.0,4818.596671,6.670255e-01,1647.172666,-4488.053788,-6.025523e+02,-4.667192e-01,4.753069e-01,3.434522e-02
6,202930 Ivezic (1998 SG172),FindOrb,500,370.0,38,59370.0008,9.997748e-07,0.0,0.000280,1.257969e-08,0.000189,-0.000205,-1.919551e-05,1.233878e-08,-2.435919e-09,2.631903e-10
7,202930 Ivezic (1998 SG172),FindOrb,500,370.0,38,59370.0008,9.997748e-07,10.0,11.723568,8.862882e-04,8.384861,2.115210,7.915936e+00,8.037389e-04,-3.642668e-04,-8.258437e-05
8,202930 Ivezic (1998 SG172),FindOrb,500,370.0,38,59370.0008,9.997748e-07,100.0,213.873302,1.842752e-02,74.523251,191.674991,-5.872625e+01,6.162375e-03,1.726862e-02,-1.842067e-03


## Varying Observatory Codes
As mentioned, one can iterate over different observatory codes by passing a list of MPC observatory code strings. Here we use the Geocenter (500) and Gemini South Observatory (I11) codes.

In [12]:
obs = ['500','I11']
runTest(ivezic,obs,dts[0],astrometric_error=errors[0],backend=FINDORB())

,orbit_id,integrator,observatory_code,arc_length [days],num_obs,epoch [mjd],delta epoch [mjd],astrometric_error [mas],delta r [km],delta v [m/s],delta x [km],delta y [km],delta z [km],delta vx [m/s],delta vy [m/s],delta vz [m/s]
0,202930 Ivezic (1998 SG172),FindOrb,500,29.0,30,59029.0008,9.997748e-07,0.0,0.000046,1.589317e-08,0.000021,-0.000040,-0.000005,-8.887649e-09,1.307467e-08,1.629640e-09
1,202930 Ivezic (1998 SG172),FindOrb,I11,29.0,30,59029.0008,9.997748e-07,0.0,0.000614,6.318958e-08,-0.000282,0.000541,0.000066,4.635154e-08,-4.256100e-08,-5.746260e-09


Wonderful! You have successfully used the tester. 

This tester can also be used for multiple asteroid orbits and the outputs will scale accordingly. For the sake of not knowing how much computing power and time the user has, I will not be demonstrating this feature here. Try running the `orbits` object above in `runTest()` for a demo. **NOTE: If you are using Binder for this demo, I do not recommend doing this as you will risk running out of memory.**

---
## Command and Bash Script Output
You can use the `out` parameter to save all the files generated by the tester to a provided path location.

---

## Integrator Specific Functionality
Integrators like Open Orb and OrbFit have specific functionality the user can tweak for testing.

### OpenOrb `multi_ranging`
This integrators `multi_ranging` feature is well explained in the wiki. Here is a short demonstration on how to use them. The multi_ranging options are as follows:
* 0 for no multi_ranging, all observations are given for initial fit. (Can be unstable for long arcs, more stable for short arcs)
* 1 for basic multi_ranging, observations are split up into groups of 3-4, then fit separately. (Somewhat stable for longer arcs)
* 2 for OrbFit style ranging, the first, middle, and last observations are only given to the fitter. (Somewhat stable for longer arcs)

It is set at option 1 by default. To change the option, you must include `multi_ranging=` and the corresponding option number to the `PYOORB()` backend function call. For example,

In [15]:
# Setting the multi_ranging setting to 2 (OrbFit style multi_ranging)
runTest(ivezic,'500',dts[0],backend=PYOORB(multi_ranging=2))

,orbit_id,integrator,observatory_code,arc_length [days],num_obs,epoch [mjd],delta epoch [mjd],astrometric_error [mas],delta r [km],delta v [m/s],delta x [km],delta y [km],delta z [km],delta vx [m/s],delta vy [m/s],delta vz [m/s]
0,202930 Ivezic (1998 SG172),OpenOrb,500,29.0,30,59015.0,0.0,0,5.787959,0.002428,2.89069,-4.975498,-0.623545,-0.001293,0.002044,0.000208


### OrbFit `arc_limit`
The `ORBFIT()` backend comes with a feature to make sure its Initial Orbit Determination (IOD) function does not give bad fits by choosing to only fit the first few observations if given a long-term arc. If the observational arc exceeds the limit, only the first months worth of observations (<31 days) or the first 8 are given to IOD. The default limit is 31 days. To change this option, you must include `arc_limit=` and the number of days you want for the limit to the `OPENORB()` backend function call. For example,

In [14]:
# Setting the arc_limit to 31 days
runTest(ivezic,'500',dts[0],backend=ORBFIT(arc_limit=31))

,orbit_id,integrator,observatory_code,arc_length [days],num_obs,epoch [mjd],delta epoch [mjd],astrometric_error [mas],delta r [km],delta v [m/s],delta x [km],delta y [km],delta z [km],delta vx [m/s],delta vy [m/s],delta vz [m/s]
0,202930 Ivezic (1998 SG172),OrbFit,500,29.0,30,59014.992,0.0,0,31.003604,0.088761,-16.40667,26.20539,2.306993,0.04385,-0.076605,-0.009345
